# Bitcoin Price Trend Analysis and Prediction

This notebook analyzes Bitcoin price trends using historical data from Yahoo Finance and predicts future prices using Holt-Winters, Prophet, and LSTM methods. The analysis includes:
- **Timeframes**: 1 Year (1Y) and 5 Years (5Y)
- **Prediction Horizons**: 1 Month (30 days) and 1 Year (365 days)
- **Prediction Methods**: Holt-Winters (Exponential Smoothing), Prophet (Additive Model), and LSTM (Long Short-Term Memory)
- **Features**: Historical prices, Short and Long Simple Moving Averages (SMAs), Predicted prices with 95% Confidence Intervals

Each section below generates a static Bokeh plot for a specific combination of timeframe, prediction method, and horizon.

## Setup and Helper Functions

First, we import necessary libraries and define helper functions to fetch data, calculate moving averages, and predict prices.

In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
import yfinance as yf
from bokeh.models import HoverTool, ColumnDataSource, Legend, NumeralTickFormatter
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import warnings
from bokeh.io import output_notebook

# Configure Bokeh for Colab
output_notebook()

warnings.filterwarnings('ignore')

# Function to calculate moving averages based on timeframe
def calculate_moving_averages(df, timeframe):
    sma_mapping = {"1Y": (50, 200), "5Y": (200, 800)}
    short_window, long_window = sma_mapping.get(timeframe, (50, 200))
    df['sma_short'] = df['price'].rolling(window=short_window).mean().astype(float)
    df['sma_long'] = df['price'].rolling(window=long_window).mean().astype(float)
    df.fillna(method="ffill", inplace=True)
    return df, short_window, long_window

# Function to predict prices with confidence intervals
def predict_prices(df, days_ahead=30, method='holt-winters'):
    df_pred = df.copy()
    if method == 'holt-winters':
        model = ExponentialSmoothing(df_pred['price'], trend="add", seasonal="add", seasonal_periods=7)
        model_fit = model.fit()
        predictions = model_fit.forecast(steps=days_ahead)
        resid = df_pred['price'] - model_fit.fittedvalues
        ci_width = 1.96 * np.std(resid)
        lower_ci = predictions - ci_width
        upper_ci = predictions + ci_width
        future_dates = pd.date_range(start=df_pred['timestamp'].iloc[-1], periods=days_ahead + 1, freq='D')[1:]
        pred_df = pd.DataFrame({
            'timestamp': future_dates,
            'price_pred': predictions,
            'lower_ci': lower_ci,
            'upper_ci': upper_ci
        })
    elif method == 'prophet':
        df_prophet = df_pred.rename(columns={'timestamp': 'ds', 'price': 'y'})
        df_prophet['ds'] = df_prophet['ds'].dt.tz_localize(None)
        model = Prophet()
        model.fit(df_prophet)
        future = model.make_future_dataframe(periods=days_ahead)
        forecast = model.predict(future)
        forecast = forecast.iloc[-days_ahead:]
        pred_df = pd.DataFrame({
            'timestamp': forecast['ds'].values,
            'price_pred': forecast['yhat'].values,
            'lower_ci': forecast['yhat_lower'].values,
            'upper_ci': forecast['yhat_upper'].values
        })
    elif method == 'lstm':
        # Prepare data for LSTM
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(df_pred['price'].values.reshape(-1, 1))
        look_back = 60
        X = []
        for i in range(look_back, len(scaled_data)):
            X.append(scaled_data[i-look_back:i, 0])
        X = np.array(X)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build and train LSTM model
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=True, input_shape=(look_back, 1)))
        model.add(LSTM(units=50))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X, scaled_data[look_back:], epochs=20, batch_size=32, verbose=0)

        # Make predictions
        last_sequence = scaled_data[-look_back:]
        predictions = []
        for _ in range(days_ahead):
            sequence = last_sequence.reshape((1, look_back, 1))
            next_pred = model.predict(sequence, verbose=0)
            predictions.append(next_pred[0,0])
            last_sequence = np.append(last_sequence[1:], next_pred[0,0])

        # Inverse transform predictions
        predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

        # Calculate confidence intervals (using residual standard deviation)
        fitted = model.predict(X, verbose=0)
        fitted = scaler.inverse_transform(fitted)
        resid = df_pred['price'].values[look_back:] - fitted.flatten()
        ci_width = 1.96 * np.std(resid)
        lower_ci = predictions.flatten() - ci_width
        upper_ci = predictions.flatten() + ci_width

        future_dates = pd.date_range(start=df_pred['timestamp'].iloc[-1], periods=days_ahead + 1, freq='D')[1:]
        pred_df = pd.DataFrame({
            'timestamp': future_dates,
            'price_pred': predictions.flatten(),
            'lower_ci': lower_ci,
            'upper_ci': upper_ci
        })
    return pred_df

# Function to fetch Bitcoin data from Yahoo Finance
def fetch_bitcoin_data(period):
    try:
        btc = yf.Ticker("BTC-USD")
        df = btc.history(period=period, interval="1d")
        if not df.empty:
            df = df.reset_index()
            df = df[['Date', 'Close']].rename(columns={'Date': 'timestamp', 'Close': 'price'})
            df['price'] = df['price'].astype(float)
            return df
    except Exception as e:
        print(f"Error fetching data from Yahoo Finance: {e}")
    return pd.DataFrame()

## 1Y Timeframe with Holt-Winters (1 Month Prediction)

This section fetches 1-year Bitcoin price data, calculates 50-day and 200-day SMAs, and predicts prices for the next 30 days using Holt-Winters exponential smoothing.

In [2]:
df_1y = fetch_bitcoin_data("1y")
df_1y, short_window, long_window = calculate_moving_averages(df_1y, "1Y")
pred_df_1m_holt = predict_prices(df_1y, days_ahead=30, method='holt-winters')
combined_df = pd.concat([df_1y, pred_df_1m_holt], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with Holt-Winters Prediction (1Y, 1M)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (Holt-Winters)')
ci_x = np.concatenate([pred_df_1m_holt['timestamp'], pred_df_1m_holt['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1m_holt['upper_ci'], pred_df_1m_holt['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (Holt-Winters)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)

## 1Y Timeframe with Holt-Winters (1 Year Prediction)

This section predicts Bitcoin prices for the next 365 days using Holt-Winters exponential smoothing over a 1-year historical timeframe.

In [3]:
pred_df_1y_holt = predict_prices(df_1y, days_ahead=365, method='holt-winters')
combined_df = pd.concat([df_1y, pred_df_1y_holt], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with Holt-Winters Prediction (1Y, 1Y)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (Holt-Winters)')
ci_x = np.concatenate([pred_df_1y_holt['timestamp'], pred_df_1y_holt['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1y_holt['upper_ci'], pred_df_1y_holt['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (Holt-Winters)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)

## 1Y Timeframe with Prophet (1 Month Prediction)

This section predicts Bitcoin prices for the next 30 days using Prophet additive model over a 1-year historical timeframe.

In [4]:
pred_df_1m_prophet = predict_prices(df_1y, days_ahead=30, method='prophet')
combined_df = pd.concat([df_1y, pred_df_1m_prophet], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with Prophet Prediction (1Y, 1M)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (Prophet)')
ci_x = np.concatenate([pred_df_1m_prophet['timestamp'], pred_df_1m_prophet['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1m_prophet['upper_ci'], pred_df_1m_prophet['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (Prophet)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpavjccukd/i9emint4.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpavjccukd/5p79xa1y.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=84701', 'data', 'file=/tmp/tmpavjccukd/i9emint4.json', 'init=/tmp/tmpavjccukd/5p79xa1y.json', 'output', 'file=/tmp/tmpavjccukd/prophet_modelw2dwatk7/prophet_model-20250315034926.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:49:26 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:49:27 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


## 1Y Timeframe with Prophet (1 Year Prediction)

This section predicts Bitcoin prices for the next 365 days using Prophet additive model over a 1-year historical timeframe.

In [5]:
pred_df_1y_prophet = predict_prices(df_1y, days_ahead=365, method='prophet')
combined_df = pd.concat([df_1y, pred_df_1y_prophet], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with Prophet Prediction (1Y, 1Y)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (Prophet)')
ci_x = np.concatenate([pred_df_1y_prophet['timestamp'], pred_df_1y_prophet['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1y_prophet['upper_ci'], pred_df_1y_prophet['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (Prophet)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpavjccukd/03_w9oor.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpavjccukd/py2ds0v8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=22592', 'data', 'file=/tmp/tmpavjccukd/03_w9oor.json', 'init=/tmp/tmpavjccukd/py2ds0v8.json', 'output', 'file=/tmp/tmpavjccukd/prophet_modell913hqtp/prophet_model-20250315034928.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:49:28 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:49:29 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


## 1Y Timeframe with LSTM (1 Month Prediction)

This section predicts Bitcoin prices for the next 30 days using Long Short-Term Memory (LSTM) neural network over a 1-year historical timeframe.

In [6]:
pred_df_1m_lstm = predict_prices(df_1y, days_ahead=30, method='lstm')
combined_df = pd.concat([df_1y, pred_df_1m_lstm], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with LSTM Prediction (1Y, 1M)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (LSTM)')
ci_x = np.concatenate([pred_df_1m_lstm['timestamp'], pred_df_1m_lstm['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1m_lstm['upper_ci'], pred_df_1m_lstm['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (LSTM)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)

## 1Y Timeframe with LSTM (1 Year Prediction)

This section predicts Bitcoin prices for the next 365 days using Long Short-Term Memory (LSTM) neural network over a 1-year historical timeframe.

In [7]:
pred_df_1y_lstm = predict_prices(df_1y, days_ahead=365, method='lstm')
combined_df = pd.concat([df_1y, pred_df_1y_lstm], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with LSTM Prediction (1Y, 1Y)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (LSTM)')
ci_x = np.concatenate([pred_df_1y_lstm['timestamp'], pred_df_1y_lstm['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1y_lstm['upper_ci'], pred_df_1y_lstm['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (LSTM)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)

## 5Y Timeframe with Holt-Winters (1 Month Prediction)

This section uses a 5-year timeframe with Holt-Winters exponential smoothing to predict Bitcoin prices for the next 30 days.

In [8]:
df_5y = fetch_bitcoin_data("5y")
df_5y, short_window, long_window = calculate_moving_averages(df_5y, "5Y")
pred_df_1m_holt = predict_prices(df_5y, days_ahead=30, method='holt-winters')
combined_df = pd.concat([df_5y, pred_df_1m_holt], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with Holt-Winters Prediction (5Y, 1M)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (Holt-Winters)')
ci_x = np.concatenate([pred_df_1m_holt['timestamp'], pred_df_1m_holt['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1m_holt['upper_ci'], pred_df_1m_holt['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (Holt-Winters)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)

## 5Y Timeframe with Holt-Winters (1 Year Prediction)

This section predicts Bitcoin prices for the next 365 days using Holt-Winters exponential smoothing over a 5-year historical timeframe.

In [9]:
pred_df_1y_holt = predict_prices(df_5y, days_ahead=365, method='holt-winters')
combined_df = pd.concat([df_5y, pred_df_1y_holt], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with Holt-Winters Prediction (5Y, 1Y)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (Holt-Winters)')
ci_x = np.concatenate([pred_df_1y_holt['timestamp'], pred_df_1y_holt['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1y_holt['upper_ci'], pred_df_1y_holt['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (Holt-Winters)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)

## 5Y Timeframe with Prophet (1 Month Prediction)

This section predicts Bitcoin prices for the next 30 days using Prophet additive model over a 5-year historical timeframe.

In [10]:
pred_df_1m_prophet = predict_prices(df_5y, days_ahead=30, method='prophet')
combined_df = pd.concat([df_5y, pred_df_1m_prophet], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with Prophet Prediction (5Y, 1M)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (Prophet)')
ci_x = np.concatenate([pred_df_1m_prophet['timestamp'], pred_df_1m_prophet['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1m_prophet['upper_ci'], pred_df_1m_prophet['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (Prophet)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpavjccukd/u4omffss.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpavjccukd/au4_9men.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=84230', 'data', 'file=/tmp/tmpavjccukd/u4omffss.json', 'init=/tmp/tmpavjccukd/au4_9men.json', 'output', 'file=/tmp/tmpavjccukd/prophet_modeln1zcujnd/prophet_model-20250315035109.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:51:09 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:51:09 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


## 5Y Timeframe with Prophet (1 Year Prediction)

This section predicts Bitcoin prices for the next 365 days using Prophet additive model over a 5-year historical timeframe.

In [11]:
pred_df_1y_prophet = predict_prices(df_5y, days_ahead=365, method='prophet')
combined_df = pd.concat([df_5y, pred_df_1y_prophet], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with Prophet Prediction (5Y, 1Y)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (Prophet)')
ci_x = np.concatenate([pred_df_1y_prophet['timestamp'], pred_df_1y_prophet['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1y_prophet['upper_ci'], pred_df_1y_prophet['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (Prophet)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpavjccukd/gnivt2ya.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpavjccukd/izisdeui.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=84229', 'data', 'file=/tmp/tmpavjccukd/gnivt2ya.json', 'init=/tmp/tmpavjccukd/izisdeui.json', 'output', 'file=/tmp/tmpavjccukd/prophet_model1zel2v5w/prophet_model-20250315035110.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:51:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:51:11 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


## 5Y Timeframe with LSTM (1 Month Prediction)

This section predicts Bitcoin prices for the next 30 days using Long Short-Term Memory (LSTM) neural network over a 5-year historical timeframe.

In [12]:
pred_df_1m_lstm = predict_prices(df_5y, days_ahead=30, method='lstm')
combined_df = pd.concat([df_5y, pred_df_1m_lstm], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with LSTM Prediction (5Y, 1M)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (LSTM)')
ci_x = np.concatenate([pred_df_1m_lstm['timestamp'], pred_df_1m_lstm['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1m_lstm['upper_ci'], pred_df_1m_lstm['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (LSTM)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)

## 5Y Timeframe with LSTM (1 Year Prediction)

This section predicts Bitcoin prices for the next 365 days using Long Short-Term Memory (LSTM) neural network over a 5-year historical timeframe.

In [13]:
pred_df_1y_lstm = predict_prices(df_5y, days_ahead=365, method='lstm')
combined_df = pd.concat([df_5y, pred_df_1y_lstm], ignore_index=True)
combined_df = combined_df[['timestamp', 'price', 'sma_short', 'sma_long', 'price_pred', 'lower_ci', 'upper_ci']].fillna({'price_pred': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan})
source = ColumnDataSource(combined_df)

p = figure(x_axis_type='datetime', title='Bitcoin Price Trends with LSTM Prediction (5Y, 1Y)', width=1000, height=500)
price_line = p.line('timestamp', 'price', source=source, color='blue', line_width=2, name='Actual Price')
sma_short_line = p.line('timestamp', 'sma_short', source=source, color='green', line_width=2, line_dash='dashed', name=f'SMA {short_window}')
sma_long_line = p.line('timestamp', 'sma_long', source=source, color='red', line_width=2, line_dash='dashed', name=f'SMA {long_window}')
pred_line = p.line('timestamp', 'price_pred', source=source, color='orange', line_width=2, line_dash='dotted', name='Prediction (LSTM)')
ci_x = np.concatenate([pred_df_1y_lstm['timestamp'], pred_df_1y_lstm['timestamp'][::-1]])
ci_y = np.concatenate([pred_df_1y_lstm['upper_ci'], pred_df_1y_lstm['lower_ci'][::-1]])
ci_band = p.patch(ci_x, ci_y, color='orange', alpha=0.2, name='ci_band')

legend = Legend(items=[
    ("Actual Price", [price_line]),
    (f"SMA {short_window}", [sma_short_line]),
    (f"SMA {long_window}", [sma_long_line]),
    ("Prediction (LSTM)", [pred_line]),
    ("95% Confidence Interval", [ci_band])
], location="top_left")
p.add_layout(legend)
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format="$0,0")

historical_hover = HoverTool(renderers=[price_line], tooltips=[('Date', '@timestamp{%F}'), ('Price', '$@price{0,0.00}'), ('SMA Short', '@sma_short{0,0.00}'), ('SMA Long', '@sma_long{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
prediction_hover = HoverTool(renderers=[pred_line, ci_band], tooltips=[('Date', '@timestamp{%F}'), ('Predicted Price', '$@price_pred{0,0.00}'), ('Lower CI', '$@lower_ci{0,0.00}'), ('Upper CI', '$@upper_ci{0,0.00}')], formatters={'@timestamp': 'datetime'}, mode='vline')
p.add_tools(historical_hover, prediction_hover)

show(p)